# Downloading data to the temp folder

In [3]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip \
    -O /tmp/horse-or-human.zip

--2020-05-10 04:04:26--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 2607:f8b0:400e:c08::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘/tmp/horse-or-human.zip’

/tmp/horse-or-human 100%[===================>] 142.65M   100MB/s    in 1.4s    

2020-05-10 04:04:28 (100 MB/s) - ‘/tmp/horse-or-human.zip’ saved [149574867/149574867]



In [0]:
import os
import zipfile
localzip = '/tmp/horse-or-human.zip'
zipref=zipfile.ZipFile(localzip,mode='r')
zipref.extractall('/tmp/horse-or-human')
zipref.close()

# Modelling 

In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [0]:
train_generator = ImageDataGenerator(rescale=1/255.0)

In [14]:
train_data = train_generator.flow_from_directory(
    '/tmp/horse-or-human/',
    target_size=(300,300),
    class_mode='binary',
    batch_size=128
)

Found 1027 images belonging to 2 classes.


In [0]:
model = tf.keras.Sequential(layers=[
                                    tf.keras.layers.Conv2D(8,(3,3),activation=tf.nn.relu,input_shape=(300,300,3)),
                                    tf.keras.layers.MaxPool2D((2,2)),
                                    tf.keras.layers.Conv2D(16,(3,3),activation=tf.nn.relu,input_shape=(300,300,3)),
                                    tf.keras.layers.MaxPool2D((2,2)),
                                    tf.keras.layers.Conv2D(32,(3,3),activation=tf.nn.relu,input_shape=(300,300,3)),
                                    tf.keras.layers.MaxPool2D((2,2)),
                                    tf.keras.layers.Conv2D(64,(3,3),activation=tf.nn.relu,input_shape=(300,300,3)),
                                    tf.keras.layers.MaxPool2D((2,2)),
                                    tf.keras.layers.Conv2D(64,(3,3),activation=tf.nn.relu,input_shape=(300,300,3)),
                                    tf.keras.layers.MaxPool2D((2,2)),
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(512,activation=tf.nn.relu),
                                    tf.keras.layers.Dense(256,activation=tf.nn.relu),
                                    tf.keras.layers.Dense(128,activation=tf.nn.relu),
                                    tf.keras.layers.Dense(1,activation=tf.nn.sigmoid)
])

In [19]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 298, 298, 8)       224       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 149, 149, 8)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 147, 147, 16)      1168      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 73, 73, 16)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 71, 71, 32)        4640      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 35, 35, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 33, 33, 64)       

In [0]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),loss="binary_crossentropy",metrics=['accuracy'])

In [21]:
model.fit_generator(train_data,steps_per_epoch=8,epochs=15,verbose=2)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/15
8/8 - 49s - loss: 0.7448 - accuracy: 0.5176
Epoch 2/15
8/8 - 47s - loss: 0.6968 - accuracy: 0.6329
Epoch 3/15
8/8 - 41s - loss: 0.5745 - accuracy: 0.7475
Epoch 4/15
8/8 - 40s - loss: 0.6718 - accuracy: 0.7675
Epoch 5/15
8/8 - 40s - loss: 0.4841 - accuracy: 0.7909
Epoch 6/15
8/8 - 40s - loss: 0.7233 - accuracy: 0.7942
Epoch 7/15
8/8 - 40s - loss: 0.2612 - accuracy: 0.9010
Epoch 8/15
8/8 - 40s - loss: 0.1790 - accuracy: 0.9244
Epoch 9/15
8/8 - 39s - loss: 0.3977 - accuracy: 0.8409
Epoch 10/15
8/8 - 46s - loss: 0.1718 - accuracy: 0.9287
Epoch 11/15
8/8 - 47s - loss: 0.1032 - accuracy: 0.9633
Epoch 12/15
8/8 - 41s - loss: 0.2891 - accuracy: 0.9077
Epoch 13/15
8/8 - 41s - loss: 0.1014 - accuracy: 0.9555
Epoch 14/15
8/8 - 40s - loss: 0.0682 - accuracy: 0.9711
Epoch 15/15
8/8 - 41s - loss: 0.1352 - accuracy: 0.9622


In [23]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

Using TensorFlow backend.


In [29]:
uploaded = files.upload()
for fn in uploaded.keys():
  img = image.load_img('/content/'+fn,target_size=(300,300))
  x=image.img_to_array(img)
  x=np.expand_dims(x,axis=0)
  c=model.predict(x)
  print(c[0])
  if(c[0]>0.5):
    print('Human')
  else:
    print('Horse')

Saving human(1).jpg to human(1).jpg
[1.]
Human
